<a href="https://colab.research.google.com/github/FB-Decipherer/Experiment-1/blob/main/Template_Match_1623_v53.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Template Match 1623
Classifier Emulation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install filetype

In [ ]:
import os
from   pathlib import Path
from   google.colab import files
from google.colab.patches import cv2_imshow

import filetype
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
content_root = '/content/drive/MyDrive/Template Match/'

targets_dir = content_root + 'Targets/'
templates_dir = content_root + 'Templates/'

In [ ]:
#@title Define Local Utility Routines:

def obj_exists(name,obj=None):
 if obj is None: return locals().has_key(name)
 else: return hasattr(obj,name)

def image_valid(image_path):

  if filetype.is_image(image_path):
    return True
  else:
    print(f"{image_path} is a not a valid image...")
    return False
  

In [ ]:
#@title Read and Display Target (larger) Image file:

# target_file_path = '/content/drive/MyDrive/Template Match/Targets/default target image a.jpeg'
target_file_path ='/content/drive/MyDrive/Template Match/Targets/Bodlein TC Prologue 1623.jpg'
# target_file_path ='/content/drive/MyDrive/Template Match/Targets/Prologue Line 7.jpg'

if(image_valid(target_file_path)):

  target_image = cv2.imread(target_file_path)

  fig = plt.figure(figsize=(80,4))
  plt.title('Original Target Image File')
  plt.imshow(cv2.cvtColor(target_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
#@title Read and Display Template (smaller) Image file:

template_file_path = '/content/drive/MyDrive/Template Match/Templates/v.jpg'

if(image_valid(template_file_path)):

  template_image = cv2.imread(template_file_path, 0)

  fig = plt.figure(figsize=(80,4))
  plt.title('Original Template Image File')
  plt.imshow(cv2.cvtColor(template_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
#@title Preprocess the Target image data:

def preprocess_target(target_image):

  target_gray = cv2.cvtColor(target_image,cv2.COLOR_BGR2GRAY)
  #target_blur = cv2.GaussianBlur(target_gray, (5,5), 0)
  #target_thresh = cv2.threshold(target_blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
  #kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (35,35))
  #target_dilate = cv2.dilate(target_thresh, kernel, iterations=2)
  pre_processed_target = target_gray

  return pre_processed_target

pre_processed_target = preprocess_target(target_image)



In [ ]:
#@title Postprocess the Results:

def post_process_results(results):


  # Suppose we are searching for an object which has multiple occurrences:
  # In that case, we will use thresholding. 
  # Let's make the threshold 0.7
  threshold = 0.70

  #@title Filter out duplicate bounding boxes:

  unfiltered_points = np.where(results >= threshold) #returns a tuple
  #print(loc)

  x = unfiltered_points[0]
  y = unfiltered_points[1]

  x_filter = []
  i = 0
  prev_x = 0
    
  # Iterating using while loop
  while i < len(x):
      diff_x = x[i] - prev_x
      #print(diff_x)

      if diff_x < 10:
        x_filter.append(False)

      else:
        #print('loc x: ' + str(x[i]))
        x_filter.append(True)

      prev_x = x[i]
      i += 1

  x_filtered = x[x_filter]
  y_filtered = y[x_filter]

  #print(x_filtered)
  #print(y_filtered)

  filtered_points = (x_filtered, y_filtered)

  return filtered_points


In [ ]:
#@title Apply the template matching onto the Target from the Template:

# Find contours, obtain bounding box coordinates, and extract ROI:
#cnts = cv2.findContours(pre_processed_target, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#cnts = cnts[0] if len(cnts) == 2 else cnts[1]# Get the shape of the template:

w, h = template_image.shape[::-1]

# Remember, that the size of the template should 
# be as much as possible the same, as its size in the target image.

results = cv2.matchTemplate(pre_processed_target,
                        template_image,
                        cv2.TM_CCOEFF_NORMED)

min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(results) 


In [ ]:
processed_results = post_process_results(results)


In [ ]:
dr = pd.DataFrame(processed_results)
dr_transposed = dr.T # or df2.transpose()
dr_transposed.columns =['x', 'y']

styles = [dict(selector="caption",
                       props=[("text-align", "center"),
                              ("font-size", "150%"),
                              ("color", 'black')])]
dr_transposed.style.set_caption("caption set").set_table_styles(styles)
dr_transposed

In [ ]:
#print("Show positions of objects in the Target which match the Template at or above the Threshold:")

for pt in zip(*unfiltered_points[::-1]):
    cv2.rectangle(target_image, pt, (pt[0] + w, pt[1] + h), (255,0,255), 1)
    confidence = results[pt[1]][pt[0]]

cv2_imshow(template_image)
cv2_imshow(target_image)



**Draw Rectanlges over the Target Image**

In [ ]:
loc_list = []
for pt in zip(*filtered_points[::-1]):
    cv2.rectangle(target_image, pt, (pt[0] + w, pt[1] + h), (255,0,255), 1)
    confidence = results[pt[1]][pt[0]]
    loc_list.append(confidence)

#print(loc_list)

dff = pd.DataFrame(loc_list, columns =['Confidence'], dtype = float) 
dff.index = dff.index + 1
print(dff) 

